### **SDXL + Prompt Engineering**

In [1]:
import os
import ast
from PIL import Image
from datasets import load_dataset
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
DTYPE_FOR_TENSORS = torch.bfloat16  

SDXL 모델 로드

In [2]:
model_id_or_path = "stabilityai/stable-diffusion-xl-base-1.0"
scheduler = DDIMScheduler.from_pretrained(model_id_or_path, subfolder="scheduler")

print("SDXL bfloat16 로드")
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id_or_path,
    scheduler=scheduler,
    torch_dtype=DTYPE_FOR_TENSORS,
    variant="fp16",
    use_safetensors=True
).to(device)

pipe.enable_vae_tiling()
pipe.enable_model_cpu_offload()

SDXL bfloat16 로드


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


편집 프롬프트 생성 함수

In [3]:
def create_editing_prompt(source_prompt, editing_dict):
    # 1. 원본 프롬프트 정리
    prompt = source_prompt.replace("[", "").replace("]", "")
    
    # 2. 편집 적용
    for edit_word, data in editing_dict.items():
        action = data["action"]
        position = data["position"]
        
        if action == "+":
            words = prompt.split()
            words.insert(position, edit_word)
            prompt = " ".join(words)
        elif action == "-":
            prompt = prompt.replace(f" {edit_word} ", " ", 1).strip()
        else:
            old_word = action
            prompt = prompt.replace(old_word, edit_word, 1)
    
    # 3. SDXL 품질 강화
    prompt += ", highly detailed, photorealistic, 8k, sharp focus"
    
    return prompt

SDXL 편집 생성 함수

In [4]:
def sdxl_edit_generate(source_prompt, editing_dict, seed=42, idx=0):
    target_prompt = create_editing_prompt(source_prompt, editing_dict)
    
    generator = torch.Generator(device).manual_seed(seed + idx)
    
    print(f"원본: {source_prompt[:40]}...")
    print(f"타겟: {target_prompt[:60]}...")
    
    image = pipe(
        prompt=target_prompt,
        negative_prompt="blurry, lowres, distorted, ugly, deformed, bad anatomy",
        num_inference_steps=28,
        guidance_scale=7.5,
        width=1024,
        height=1024,
        generator=generator
    ).images[0]
    
    return image, target_prompt

In [5]:
print("PIE-Bench++ 데이터셋 로드...")
hf_dataset = load_dataset("UB-CVML-Group/PIE_Bench_pp", "0_random_140", split="V1")

TARGET_INDICES = [0, 2, 3, 13, 25, 40, 48, 58, 68, 102, 135]

success_count = 0

for idx in TARGET_INDICES:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    try:
        sample = hf_dataset[idx]
        
        # 편집 정보 파싱
        edit_action = ast.literal_eval(sample['edit_action']) if isinstance(sample['edit_action'], str) else sample['edit_action']
        
        image, final_prompt = sdxl_edit_generate(
            sample['source_prompt'], 
            edit_action, 
            seed=42, 
            idx=idx
        )
        
        # 저장
        image.save(f"sdxl_{idx}.jpg")
        print(f"sdxl_{idx}.jpg 생성")
        success_count += 1
        
    except Exception as e:
        print(f"오류: {str(e)[:60]}")
        continue


PIE-Bench++ 데이터셋 로드...
원본: a slanted mountain bicycle on the road i...
타겟: a slanted rusty mountain motorcycle in front of a fence, hig...


  0%|          | 0/28 [00:00<?, ?it/s]

sdxl_0.jpg 생성
원본: a cat sitting on a wooden chair...
타겟: a red with flowers in mouth dog standing on a metal chair, h...


  0%|          | 0/28 [00:00<?, ?it/s]

sdxl_2.jpg 생성
원본: blue light, a black and white cat is pla...
타겟: blue light, a black and white dog is playing with a yellow b...


...

sdxl_68.jpg 생성
원본: a rose on the rail...
타겟: a white tulip on the rail, highly detailed, photorealistic, ...


  0%|          | 0/28 [00:00<?, ?it/s]

sdxl_102.jpg 생성
원본: the two people are standing on rocks wit...
타겟: the two old people are sitting on boat with a fish, highly d...


  0%|          | 0/28 [00:00<?, ?it/s]

sdxl_135.jpg 생성
